In [48]:
#HF Example

import torch
from transformers import pipeline

pipe = pipeline(
    task="image-classification",
    model="facebook/dinov2-small-imagenet1k-1-layer",
    dtype=torch.float16,
    device="cuda"
)

pipe("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg")

/home/adam/anaconda3/envs/CLIP/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


[{'label': 'tiger, Panthera tigris', 'score': 0.24335609376430511},
 {'label': 'tiger cat', 'score': 0.24146227538585663},
 {'label': 'lynx, catamount', 'score': 0.16084855794906616},
 {'label': 'marmot', 'score': 0.043973542749881744},
 {'label': 'tabby, tabby cat', 'score': 0.03293466567993164}]

In [ ]:


from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
cls_embedding = last_hidden_states[:, 0, :]



In [50]:
print(cls_embedding.size())

torch.Size([1, 768])


In [40]:
file=hdf5.open_HDF5("downloaded_embeddings.h5")
vectors=file["vectors"]
vectors=vectors[:] #load into a np array
labels=file["coordinates"]
labels=labels[:]
print(vectors)
print(labels)
file.close()

[[-3.8935735   0.5611535   0.7221834  ... -1.6112721  -1.7305301
  -1.24625   ]
 [ 0.959288    1.5554346  -0.45038044 ...  0.38674653 -1.9512769
   0.13411409]
 [ 0.90353054  4.137877    1.1482705  ... -3.668721    0.68040836
   1.5894419 ]
 ...
 [ 1.7816428  -0.3815659   0.29235205 ... -0.32355672  0.05422599
   0.9596687 ]
 [-1.0558149   0.5838056  -1.5675911  ... -1.1514442   0.01467256
  -1.4091729 ]
 [ 0.10558449  1.7960936  -1.2238214  ... -0.98181576  2.1069787
  -1.3743852 ]]
[[48.284294  4.02083 ]
 [37.122986 -3.568118]
 [52.24937   8.912947]
 ...
 [44.60789  11.12855 ]
 [50.534054 -3.564082]
 [50.379196  4.45012 ]]


In [ ]:
import h5py
import hdf5
#Size of hp5
print("agah")
path="/home/adam/source/CLIP/full_dataset_embeddings.h5"
hdf5.get_size(path)




coordinates: shape=(3167055, 2), dtype=float32
vectors: shape=(3167055, 768), dtype=float32


In [ ]:
#Test the clip model


import utils
import importlib
import torch.nn as nn
import torch
importlib.reload(utils)
from tqdm import tqdm

#for having the right model; print the model if unsurea about the layers
dim_fourier_encoding=64 #multiple of 4!!
dim_hidden=256
dim_emb=128 #this one is actually shared with img embeddings
device="cuda"
data_path="/home/adam/source/CLIP/full_dataset_embeddings.h5"
#dim image layer size: 
#As of now: linear from 768 to dim_emb. We could also have MLP if non-linearity needed

image_encoder=nn.Linear(768,dim_emb).to(device)
pos_encoder=utils.Fourier_MLP(original_dim=2, fourier_dim=dim_fourier_encoding, hidden_dim=dim_hidden, output_dim=dim_emb).to(device)

model= utils.DoubleNetwork(image_encoder,pos_encoder).to(device)
model.load_state_dict(torch.load("/home/adam/source/CLIP/save_14_to_17-10-25/model.pt", weights_only=True))
model.eval()

nbr_iter=500000
mean_sim, mean_asim, std_sim, std_asim = utils.test_similarity(data_path,model, nbr_samples=2,device="cuda",nbr_iter=nbr_iter,plot=True)
print("Mean:", mean_sim, mean_asim)
print("Std:", std_sim, std_asim)
#for the whole test  set:
#Mean: tensor(0.1810, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.0827, device='cuda:0', grad_fn=<DivBackward0>)
#Std: tensor(0.0645, device='cuda:0', grad_fn=<SqrtBackward0>) tensor(0.0726, device='cuda:0', grad_fn=<SqrtBackward0>)

In [5]:
#Number of occurences in Switwerland

import pandas as pd

# Replace 'data.tsv' with the path to your file
file_path = '/home/adam/source/CLIP/data_plantnet_obsevations/occurrence.txt'

# Load the TSV file
df = pd.read_csv(file_path, sep='\t', low_memory=False)

# Check the column names for country info
print(df.columns)

# Filter rows where the country is Switzerland
switzerland_data = df[df['countryCode'] == 'CH']

# Count the number of rows
num_datapoints = len(switzerland_data)

print(f"Number of datapoints in Switzerland: {num_datapoints}")

# Filter rows where the country is Switzerland
france_data = df[df['countryCode'] == 'FR']

# Count the number of rows
num_datapoints = len(france_data)

print(f"Number of datapoints in France: {num_datapoints}")


Index(['gbifID', 'accessRights', 'bibliographicCitation', 'language',
       'license', 'modified', 'publisher', 'references', 'rightsHolder',
       'type',
       ...
       'publishedByGbifRegion', 'level0Gid', 'level0Name', 'level1Gid',
       'level1Name', 'level2Gid', 'level2Name', 'level3Gid', 'level3Name',
       'iucnRedListCategory'],
      dtype='object', length=226)
Number of datapoints in Switzerland: 74963
Number of datapoints in France: 884128


densité de points en suisse: 
74963/41 285 km2=1.81

France: 884128/632 702 km2=1.39
We good

In [9]:
# Replace 'data.tsv' with the path to your file
file_path =  '/home/adam/source/CLIP/data_plantnet_obsevations/occurrence.txt'

# Load the TSV file
df = pd.read_csv(file_path, sep='\t', low_memory=False)

# Count datapoints per country
country_counts = df['countryCode'].value_counts()

# Display results
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Avoid line wrapping
pd.set_option('display.max_colwidth', None)  # Full column width
print(country_counts)


countryCode
FR    884128
ES    253748
DE    237009
IT    199536
GB    132793
US    125428
NL    104618
CH     74963
CZ     61113
BE     59173
AT     44565
PL     36438
CA     29717
PT     28453
RU     22644
HU     21479
SK     16528
BR     15615
GR     14699
KE     14308
SE     13236
HR     11800
TR     11083
IN     11004
IE      9793
DK      9445
NO      9042
UA      8886
RO      8085
FI      8053
SI      7181
RE      6587
AR      6290
AU      4955
BG      4680
CL      3733
MX      3678
RS      3487
CR      3089
MA      3013
DZ      2681
CO      2623
GP      2530
LU      2396
ZA      2232
EE      2055
MQ      1981
BA      1895
ID      1850
JP      1669
IL      1645
IR      1634
BY      1602
ME      1554
LV      1549
LT      1506
NC      1490
IS      1312
TH      1302
MG      1248
CG      1143
NZ      1088
CY      1075
GE       996
MT       969
DO       946
AL       825
PE       783
GF       741
UY       719
MY       712
VN       676
AD       672
CN       668
TN       656
EC       651


In [ ]:
from pyproj import Proj, transform

def CH_to_coord(east, north): 
    '''Mind the order! It swaps to respect both order conventions
    ETH doc: https://ia.arch.ethz.ch/lat-lon-to-ch-coordinates/'''

    pWorld = Proj(init="epsg:4326")
    pCH = Proj(init="epsg:2056")
    lon,lat, _=transform(pCH,pWorld, east,north,0)
    return lat,lon

#Bern: 2,600,000 m (east) and N = 1,200,000 m (north).
east=2600000
north=1200000
e=46
n=7
result=CH_to_coord(east, north)
print(result)

(46.95108277187109, 7.43863242087181)


/home/adam/anaconda3/envs/CLIP/lib/python3.13/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/adam/anaconda3/envs/CLIP/lib/python3.13/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_1217216/2512154143.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon,la

In [2]:
#Get the Model
import utils
import importlib
import torch.nn as nn
import torch
importlib.reload(utils)
from tqdm import tqdm
dim_fourier_encoding=64 #multiple of 4!!
dim_hidden=256
dim_emb=128 #this one is actually shared with img embeddings
device="cuda"
data_path="/home/adam/source/CLIP/full_dataset_embeddings.h5"
#dim image layer size: 
#As of now: linear from 768 to dim_emb. We could also have MLP if non-linearity needed

image_encoder=nn.Linear(768,dim_emb).to(device)
pos_encoder=utils.Fourier_MLP(original_dim=2, fourier_dim=dim_fourier_encoding, hidden_dim=dim_hidden, output_dim=dim_emb).to(device)

model= utils.DoubleNetwork(image_encoder,pos_encoder).to(device)
model.load_state_dict(torch.load("/home/adam/source/CLIP/save_14_to_17-10-25/model.pt", weights_only=True))
model.eval()

DoubleNetwork(
  (image_encoder): Linear(in_features=768, out_features=128, bias=True)
  (pos_encoder): Fourier_MLP(
    (MLP): CustomMLP(
      (lin1): Linear(in_features=64, out_features=256, bias=True)
      (relu1): ReLU()
      (lin2): Linear(in_features=256, out_features=256, bias=True)
      (relu2): ReLU()
      (lin3): Linear(in_features=256, out_features=128, bias=True)
    )
  )
)

In [ ]:
#Fit model

from sklearn.linear_model import PoissonRegressor
import pandas as pd
import numpy as np
import utils
import importlib
import torch
importlib.reload(utils)

pd.set_option('display.max_columns', None)   # show all columns
pd.set_option('display.width', None)         # don't wrap lines
pd.set_option('display.max_colwidth', None)  # show full column content if needed
#np.set_printoptions(threshold=np.inf)      # print entire array, no truncation
#np.set_printoptions(linewidth=np.inf)     


po_data=pd.read_csv("data_SDM_NCEAS/SWItrain_po.csv")

y = pd.get_dummies(po_data["spid"])
po_data = pd.concat([po_data.drop(columns=["spid"]), y], axis=1) #For the full one-hot-encoded dataframe

po_covariates=po_data.loc[:, ["bcc","calc","ccc","ddeg","nutri","pday","precyy","sfroyy","slope","sradyy","swb","tavecc","topo"]]

X_cov=po_covariates.to_numpy()
y=y.to_numpy()

def fit_multi_GLM(X,y):
    PR_list=[]
    for i in tqdm(range(y.shape[1])):
        PR=PoissonRegressor()
        PR.fit(X,y[:,i])
        PR_list.append(PR)
    return PR_list

'''def get_embeddings(df,pos_encoder, dim_emb=128,device="cuda"):
    X = []
    for idx, row in df.iterrows():
        lat, lon= utils.CH_to_coord(df["y"],df["x"]) #Swiss epsg:21781 convention on the order
        if idx % 1000==0:
            print("lat", lat)
            print("lon", lon)
        coord = torch.tensor([[lat, lon]], dtype=torch.float32).to(device)
        embedding = pos_encoder(coord)
        X.append(embedding.cpu().detach().numpy())
    return X'''

from pyproj import Transformer

transformer_CH_to_WGS = Transformer.from_crs(
    "EPSG:21781", "EPSG:4326", always_xy=True
)

def get_embeddings(df, pos_encoder, device="cuda"):
    # Vectorized coordinate conversion
    lons, lats = transformer_CH_to_WGS.transform(
        df["x"].values, df["y"].values
    )

    coords = torch.tensor(
        np.column_stack([lats, lons]),
        dtype=torch.float32
    ).to(device)

    with torch.no_grad():
        emb = pos_encoder(coords).cpu().numpy()

    return emb



print("getting embeddings")
X_emb=get_embeddings(po_data, model.pos_encoder)
print("shape Xemb", X_emb.shape)
print("shape Xcov", X_cov.shape)
print("shape y",y.shape)
#Fit GLM
print("fitting models")

PR_emb=fit_multi_GLM(X_emb,y)
PR_cov=fit_multi_GLM(X_cov,y)


y


getting embeddings
shape Xemb (35105, 128)
shape Xcov (35105, 13)
shape y (35105, 30)
fitting models


  0%|          | 0/30 [00:00<?, ?it/s]/home/adam/anaconda3/envs/CLIP/lib/python3.13/site-packages/sklearn/linear_model/_glm/glm.py:286: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result(
/home/adam/anaconda3/envs/CLIP/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:330: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/adam/anaconda3/envs/CLIP/lib/python3.13/site-packages/sklearn/linear_model/_glm/glm.py:286: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergenc

array([[ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True]], shape=(35105, 30))

In [ ]:
#PA data
from sklearn.metrics import mean_squared_error

pa_data=pd.read_csv("data_SDM_NCEAS/SWItest_pa.csv")
env=pd.read_csv("data_SDM_NCEAS/SWItest_env.csv")

y_target = pa_data.loc[:, [
    'swi01','swi02','swi03','swi04','swi05','swi06','swi07','swi08','swi09','swi10',
    'swi11','swi12','swi13','swi14','swi15','swi16','swi17','swi18','swi19','swi20',
    'swi21','swi22','swi23','swi24','swi25','swi26','swi27','swi28','swi29','swi30'
]]
X_test = env.loc[:, [
    'bcc', 'calc', 'ccc', 'ddeg', 'nutri', 'pday', 'precyy',
    'sfroyy', 'slope', 'sradyy', 'swb', 'tavecc', 'topo'
]]
X_test=X_test.to_numpy()
y_target=y_target.to_numpy()

y_pred_cov_0=PR_cov[0].predict(X_test)
#y_pred_emb_0=PR_emb[0].predict(X_test) Not that simple

mse = mean_squared_error(y_target[:,0], y_pred_cov_0)
print("MSE:", mse)


MSE: 0.010459797034555213
